[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1R5mFGDCU27KA6tMDpr15CwlP8MCpMEJM?usp=sharing)


In [5]:
# Install agno if available on PyPI
!pip install agno
!pip install exa_py
!pip install youtube_transcript_api
!pip install python-dotenv pymupdf Pillow
!pip install --upgrade openai




# OR install directly from GitHub if needed (replace with the actual repo URL)
# !pip install git+https://github.com/your-org/agno.git


In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

# Optional: Set them as env vars if tools expect them there
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["EXA_API_KEY"] = os.getenv("EXA_API_KEY")


In [8]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.exa import ExaTools
from agno.tools.youtube import YouTubeTools

study_partner = Agent(
    name="StudyScout",  # Fixed typo in name
    model=OpenAIChat(id="gpt-4o"),
    tools=[ExaTools(), YouTubeTools()],
    markdown=True,
    description="You are a study partner who assists users in finding resources, answering questions, and providing explanations on various topics.",
    instructions=[
        "Use Exa to search for relevant information on the given topic and verify information from multiple reliable sources.",
        "Break down complex topics into digestible chunks and provide step-by-step explanations with practical examples.",
        "Share curated learning resources including documentation, tutorials, articles, research papers, and community discussions.",
        "Recommend high-quality YouTube videos and online courses that match the user's learning style and proficiency level.",
        "Suggest hands-on projects and exercises to reinforce learning, ranging from beginner to advanced difficulty.",
        "Create personalized study plans with clear milestones, deadlines, and progress tracking.",
        "Provide tips for effective learning techniques, time management, and maintaining motivation.",
        "Recommend relevant communities, forums, and study groups for peer learning and networking.",
        "Make sure that all links are valid and all youtube links are visible not deleted",
        "porvide materials links for each day"
    ],
)
study_partner.print_response(
    "I want to learn about Postgres in depth. I know the basics, have 2 weeks to learn, and can spend 3 hours daily. Please share some resources and a study plan.",
    stream=True,
)

Output()

In [9]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

exam_prep_agent = Agent(
    name="ExamPrepGPT",
    model=OpenAIChat(id="gpt-4o"),
    markdown=True,
    description="You're an AI tutor helping users study for exams from uploaded PDFs.",
    instructions=[
        "Use the provided PDF content to generate quizzes, flashcards, and summaries.",
        "Explain difficult topics simply.",
        "Encourage active recall through Q&A.",
    ]
)


In [10]:
from google.colab import files
import fitz  # PyMuPDF

uploaded = files.upload()
pdf_path = next(iter(uploaded))

def extract_pdf_text(path):
    doc = fitz.open(path)
    return "\n".join([page.get_text() for page in doc])

pdf_text = extract_pdf_text(pdf_path)
#print("✅ Extracted sample:\n", pdf_text[:500])


Saving Script.pdf to Script.pdf


In [11]:
exam_prep_agent.print_response(f"""
Based on this study material:

{pdf_text[:4000]}

Please:
1. Summarize the key concepts.
2. Generate 5 multiple-choice questions with answers.
3. Create 5 flashcards with questions and answers.
""", stream=True)


Output()

In [12]:
import openai
import base64
import io
from PIL import Image
from google.colab import files

uploaded = files.upload()
image_path = next(iter(uploaded))

# Open and encode the image as base64
image = Image.open(image_path).convert("RGB")
buffer = io.BytesIO()
image.save(buffer, format="PNG")
img_base64 = base64.b64encode(buffer.getvalue()).decode("utf-8")





Saving write-essays-handwriting.jpg to write-essays-handwriting.jpg


In [13]:
from agno.models.openai import OpenAIChat
from dotenv import load_dotenv
import os
from openai import OpenAI

# Load environment variables from .env
load_dotenv()

# Get the key from environment
api_key = os.getenv("OPENAI_API_KEY")

# Create OpenAI client using the key
client = OpenAI(api_key=api_key)
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Please evaluate this handwritten essay. Give feedback and a score like IELTS writing test out of 9."},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_base64}"}}
            ]
        }
    ]
)

# Show the result
print(response.choices[0].message.content)


Based on the handwritten essay, here's the feedback according to the IELTS writing criteria:

### Task Achievement
- The essay addresses the topic of diversity in India effectively and provides relevant examples. However, it lacks depth in exploring how diversity leads to discrimination or prejudice, which seems to be the intended continuation.

### Coherence and Cohesion
- The essay is coherent and ideas are connected logically. Transition from one idea to another is smooth, but the conclusion is missing or not apparent in the given text.

### Lexical Resource
- Good range of vocabulary is used, such as "geography," "history," and "celebrate." However, more varied language could enhance the essay.

### Grammatical Range and Accuracy
- The essay demonstrates a good command of grammar. Sentence structures are varied and mostly accurate. Some minor issues like "practised" could be more varied in passive versus active voice.

### Overall Score
I would rate this essay around **6.5 out of 9